In [1]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [15]:
# Read in data
df = pd.read_csv("exoplanet_data.csv")

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [16]:
print(df["koi_disposition"].unique())

['CONFIRMED' 'FALSE POSITIVE' 'CANDIDATE']


# Select features (columns)

In [17]:
X = df.iloc[:,1:]
y = df.iloc[:,0]

In [18]:
# Search for top 10 features according to feature importances
from sklearn.ensemble import ExtraTreesClassifier

model = ExtraTreesClassifier()
model.fit(X,y)
model.feature_importances_

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


array([0.13211439, 0.1946186 , 0.13592546, 0.05396782, 0.01259417,
       0.01471817, 0.01994641, 0.01341237, 0.01623722, 0.02147937,
       0.01102223, 0.01427836, 0.00843672, 0.01761901, 0.02866724,
       0.02736986, 0.01165098, 0.01283646, 0.01306142, 0.01189827,
       0.00892253, 0.0105421 , 0.01124173, 0.00698029, 0.00885629,
       0.00712549, 0.03562622, 0.0084843 , 0.01050244, 0.0182123 ,
       0.01543324, 0.01199693, 0.00898218, 0.01182022, 0.00721153,
       0.00876931, 0.00824683, 0.01024664, 0.00966692, 0.00927796])

In [19]:
feat_imp = pd.Series(model.feature_importances_, index=X.columns).nlargest(10)
feat_imp

koi_fpflag_ss        0.194619
koi_fpflag_co        0.135925
koi_fpflag_nt        0.132114
koi_fpflag_ec        0.053968
koi_model_snr        0.035626
koi_duration_err1    0.028667
koi_duration_err2    0.027370
koi_time0bk_err2     0.021479
koi_period_err2      0.019946
koi_steff_err1       0.018212
dtype: float64

In [20]:
# Set features based on feature importances
X = df[feat_imp.index]

# Use koi_disposition for y values
y = df['koi_disposition']

# Create a Train Test Split

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

# Pre-processing

Scale the data using the MinMaxScaler

In [22]:
from sklearn.preprocessing import MinMaxScaler

# Create scaler object
X_scaler = MinMaxScaler().fit(X_train)

# Scale training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Train the Model
Using Random Forest


In [24]:
from sklearn.ensemble import RandomForestClassifier

# Initialize model (random forest)
rf = RandomForestClassifier(n_estimators=300)

# Train the model
rf = rf.fit(X_train_scaled, y_train)

In [25]:
# Score the model
print(f"Training Data Score: {rf.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {rf.score(X_test_scaled, y_test)}")

Training Data Score: 1.0
Testing Data Score: 0.8804347826086957


# Predict

In [32]:
predictions = rf.predict(X_test_scaled)
df_pred = pd.DataFrame({"Actual":y_test, "Predicted":predictions}) 
df_pred.head()

,Actual,Predicted
6218,CANDIDATE,CANDIDATE
6961,CANDIDATE,CANDIDATE
965,CANDIDATE,CANDIDATE
2416,CONFIRMED,CONFIRMED
6533,FALSE POSITIVE,FALSE POSITIVE


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [ ]:
from sklearn.model_selection import GridSearchCV
import numpy as np

# Create the GridSearchCV model
param_grid = {'C':np.logspace(-4, 4, 20),
             'penalty':['l1','l2']}

grid = GridSearchCV(classifier, param_grid, verbose=3)

In [ ]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train)

In [ ]:
print(grid.best_params_)
print(grid.best_score_)

In [ ]:
grid_predictions = grid.predict(X_test_scaled)
df_grid = pd.DataFrame({"Actual":y_test, "Predicted":grid_predictions})
df_grid.head()

# Save the Model

In [ ]:
import joblib
filename = 'name.sav'
joblib.dump(my_model, filename)